In [1]:
import torch
import torchvision
import numpy as np
import os.path as osp
import os
import math

In [2]:
yolo_pred_path = "/home/zabulskyy/Projects/CTU-Research/yolo_predictions/extended/"

In [3]:
def read_yolo_pred(vot_class, yolo_pred_path=yolo_pred_path, pred_format="csv", first_len=True):
    yolo_pred_file = osp.join(yolo_pred_path, vot_class + "." + pred_format)
    with open(yolo_pred_file, 'r') as file:
        l =file.read().split("\n")
        l.pop(-1)
        im_len = -1
        if first_len:
            im_len = float(l.pop(0))
        l = [[float(x) for x in y.split(",")] for y in l]            
        return torch.tensor(l), im_len
    

In [4]:
yp, imlen = read_yolo_pred("birds2")

In [5]:
yp.shape

torch.Size([13018, 86])

In [6]:
torch.max(yp[:, 6:], dim=1)[0].shape

torch.Size([13018])

In [7]:
# yp = yp[torch.max(yp[:, 6:], dim=1)[0] > .999]

In [8]:
yp.shape

torch.Size([13018, 86])

In [9]:
descriptors = yp[:, 6:]

In [10]:
descriptors.shape

torch.Size([13018, 80])

In [11]:
cls_probs, classes = torch.max(descriptors, dim=1)

In [12]:
ff = yp[yp[:, 0] == 0]

In [13]:
for i in range(len(ff)):
    print(ff[i][:5], classes[i])

tensor([   0.0000,  435.1782,  384.9973,  540.7308,  458.9548]) tensor(14)
tensor([   0.0000,  192.9908,  363.3358,  312.4002,  491.7515]) tensor(14)
tensor([   0.0000,  438.0448,  382.3414,  542.4027,  446.4881]) tensor(14)
tensor([   0.0000,   66.5325,  389.9662,  170.5790,  495.1159]) tensor(14)
tensor([   0.0000,  300.3207,  362.2326,  411.3023,  463.9341]) tensor(14)
tensor([   0.0000,  208.1563,  368.3787,  322.2943,  488.3542]) tensor(14)
tensor([   0.0000,   75.2821,  387.6054,  154.1457,  499.7486]) tensor(14)
tensor([   0.0000,  436.2143,  371.9668,  540.4633,  452.4739]) tensor(14)
tensor([   0.0000,  634.3792,  244.2449,  736.1362,  487.3626]) tensor(0)
tensor([   0.0000,  307.6283,  360.9758,  391.4435,  467.1480]) tensor(14)
tensor([   0.0000,  225.8586,  362.4484,  316.3410,  493.2590]) tensor(14)
tensor([   0.0000,  435.7800,  384.6048,  536.9751,  460.7913]) tensor(14)
tensor([   0.0000,   68.1593,  386.1619,  145.9024,  500.7172]) tensor(14)
tensor([   0.0000,  296.33

In [14]:
classes[(torch.max(classes, dim=1)[0] > .9) == 1].shape, classes[(torch.max(classes, dim=1)[0] > .9) == 1]

RuntimeError: dimension out of range (expected to be in range of [-1, 0], but got 1)

In [41]:
people = ff[classes[:20] == 0]

In [42]:
people_desc = people[:, 6:]
people_desc.shape

torch.Size([6, 80])

In [43]:
dist = torch.nn.modules.PairwiseDistance()


In [44]:
people[:, :6]

tensor([[   0.0000,  634.3792,  244.2449,  736.1362,  487.3626,    0.9946],
        [   0.0000,  709.7218,  212.4494,  847.9936,  464.6613,    0.9403],
        [   0.0000,  655.0116,  249.7126,  727.3983,  482.2511,    0.7553],
        [   0.0000,  694.6251,  214.6559,  843.2198,  459.1760,    0.9816],
        [   0.0000,  696.7382,  219.4656,  838.8110,  481.0114,    0.9748],
        [   0.0000,  709.4280,  222.1625,  849.6510,  477.5515,    0.9652]])

In [46]:
kid = people[[0, 2]]

In [47]:
kid_desc = kid[:, 6:]
kid_desc.shape

torch.Size([2, 80])

In [48]:
kk = dist(kid_desc[0].view([1, -1]), kid_desc[1].view([1, -1]))

In [49]:
kp = dist(kid_desc[1].view([1, -1]), people[1][6:].view([1, -1]))

In [50]:
kid0 = kid[0]

In [55]:
people[:, torch.argmax(people[:, 6:], dim=1) + 6] = 0

In [54]:
for i in range(len(people)):
    print(dist(people[4][6:].view([1, -1]), people[i][6:].view([1, -1])).float())

tensor(1.00000e-04 *
       [ 5.7287])
tensor(1.00000e-04 *
       [ 5.2413])
tensor(1.00000e-03 *
       [ 1.2829])
tensor(1.00000e-04 *
       [ 3.9949])
tensor(1.00000e-06 *
       [ 8.9443])
tensor(1.00000e-04 *
       [ 4.2070])


In [40]:
for i in range(len(people)):
    print(float(dist(people[1][6:].view([1, -1]), people[i][6:].view([1, -1]))), "\n", people[i, 1:5].tolist(), "\n")

0.00017043043044395745 
 [634.379150390625, 244.2449493408203, 736.1361694335938, 487.3626403808594] 

8.944271939981263e-06 
 [709.7218017578125, 212.4493865966797, 847.9935913085938, 464.6612854003906] 

0.001237634103745222 
 [655.0115966796875, 249.71258544921875, 727.3982543945312, 482.2510986328125] 

0.00015421138959936798 
 [694.6250610351562, 214.6558837890625, 843.2197875976562, 459.176025390625] 

0.0005185101763345301 
 [696.7382202148438, 219.46560668945312, 838.81103515625, 481.0113525390625] 

0.00037562305806204677 
 [709.427978515625, 222.16250610351562, 849.6510009765625, 477.5514831542969] 

